In [1]:
import torch
import pandas
import numpy as np

In [2]:
import pandas
#dataset taken from https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms
    
    
def read_dataset(csv_name = 'sao-paulo-properties-april-2019.csv'):
    """
    Reads a csv dataset 
    returns it as a pytorch tensor
    """
    data_frame = pandas.read_csv(csv_name)
        
    #do data preprocessing and return torch matrix with targets in the last column
    """
    Creates a new column with categories using the "price" column
      1) 900000 < y , categoría 4
      2) 580000 < y < 900000, categoría 3
      3) 400000 < y < 580000, categoría 2
      4) y < 400000, categoría 1    
    """
    data_frame.loc[ data_frame['Price'] <  400000, 'Class'] = 1
    data_frame.loc[(data_frame['Price'] >= 400000) & (data_frame['Price'] < 580000), 'Class'] = 2
    data_frame.loc[(data_frame['Price'] >= 580000) & (data_frame['Price'] < 900000), 'Class'] = 3
    data_frame.loc[ data_frame['Price'] >= 900000, 'Class'] = 4
    data_frame['Class'] = data_frame['Class'].astype('int')
    
    #Remove all unnecesary attributes and leave only the ones that we need
    data_frame = data_frame.loc[:, ['Rooms', 'Size', 'Toilets', 'Parking', 'Class']]
    
    return data_frame

In [23]:
class Node_CART:    
    def __init__(self, num_classes = 4, ref_CART = None, current_depth = 0):
        """
        Create the node attributes
        param num_classes: K number of classes to classify
        param ref_cart: reference to the tree containing the node
        param current_depth: current depth of the node in the tree
        """
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None
        self.accuracy_dominant_class = None        
        self.num_classes = num_classes
        self.current_depth = current_depth
    
    def to_xml(self, current_str = ""):
        """
        Recursive function to write the node content to an xml formatted string
        param current_str : the xml content so far in the whole tree
        return the string with the node content
        """
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>" 
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if(self.node_right != None):
            str_left = self.node_right.to_xml(current_str)
            str_node += str_left
        if(self.node_left != None):
            str_right = self.node_left.to_xml(current_str)
            str_node += str_right
            
        if(self.is_leaf()):
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node
    
    def is_leaf(self):
        """
        Checks whether the node is a leaf
        """
        return (self.node_left == None and self.node_right == None)
    
    def create_with_children(self, data_torch, current_depth, list_selected_features = [], min_gini = 0.000001):
        """
        Creates a node by selecting the best feature and threshold, and if needed, creating its children
        param data_torch: dataset with the current partition to deal with in the node
        param current_depth: depth counter for the node
        param list_selected_features: list of selected features so far for the CART building process
        param min_gini: hyperparmeter selected by the user defining the minimum tolerated gini coefficient for a  node
        return the list of selected features so far
        """        
        #update depth of children
        depth_children = current_depth + 1
        if(depth_children <= self.ref_CART.get_max_depth()):
            num_observations = data_torch.shape[0]            
            #careful with max depth
            #if no threshold and feature were selected, select it using a greedy approach            
            (threshold_value, feature_num, gini) = self.select_best_feature_and_thresh(data_torch, list_features_selected = list_selected_features)
            list_selected_features += [feature_num]
            #store important data in attributes
            self.threshold_value = threshold_value
            self.feature_num = feature_num
            self.data_torch_partition = data_torch
            self.gini = gini            
            num_features = data_torch.shape[1]
            #data_torch_left = torch.zeros(1, num_features)
            #data_torch_right = torch.zeros(1, num_features)
            #create the right and left node data if the current gini is still high            
            if(self.gini > min_gini):
                print(feature_num)
                data_torch_left = data_torch[data_torch[feature_num] < threshold_value]
                data_torch_right = data_torch[data_torch[feature_num] >= threshold_value]
                #if the new partitions have more than min_observations, make them
                if(data_torch_left.shape[0] >= self.ref_CART.get_min_observations() and data_torch_right.shape[0] >= self.ref_CART.get_min_observations()):
                    #add data to the right and left children
                    self.node_right = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    self.node_left = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    list_selected_features = self.node_right.create_with_children(data_torch_right, depth_children, list_selected_features = list_selected_features)            
                    self.node_left.create_with_children( data_torch_left, depth_children, list_selected_features = list_selected_features)
        #if is leaf, fill the         
        if(self.is_leaf()):
            print("Is leaf /n", data_torch)
            labels_data = data_torch[:, -1]
            self.dominant_class = torch.mode(labels_data).values.item()
            num_obs_label = labels_data[labels_data == self.dominant_class].shape[0]
            self.accuracy_dominant_class = num_obs_label / labels_data.shape[0]           
            
        return list_selected_features
    
    
    def select_best_feature_and_thresh(self, data_torch, list_features_selected = [], num_classes = 4):
        """
        ONLY USE  2 FORS
        Selects the best feature and threshold that minimizes the gini coefficient
        param data_torch: dataset partition to analyze
        param list_features_selected list of features selected so far, thus must be ignored 
        param num_classes: number of K classes to discriminate from 
        return min_thresh, min_feature, min_gini found for the dataset partition when 
        selecting the found feature and threshold
        """       
        
        #TODO
        #Tenemos que reccorer todos los features y para cada feature todos los values posibles y sacar los ginis
        #return selected cut
        min_thresh = 0
        min_feature = ''
        min_gini = 1
    
        for feature_name in data_torch.iloc[: , :4]:
            current_feature_index = data_torch.columns.get_loc(feature_name)
            for tresh in data_torch[feature_name].unique():
                print(feature_name, current_feature_index, tresh)
                data_torch_left = data_torch[data_torch[feature_name] < tresh]
                data_torch_right = data_torch[data_torch[feature_name] >= tresh]
                #print(data_torch_left, data_torch_right)
                
                gini_left = self.calculate_gini(data_torch_left, feature_name)
                gini_right = self.calculate_gini(data_torch_right, feature_name)
                gini_ponderado = ((data_torch_left.shape[0] / data_torch.shape[0]) * gini_left) + ((data_torch_right.shape[0] / data_torch.shape[0]) * gini_right)
                
                if gini_ponderado < min_gini:
                    min_gini = gini_ponderado
                    min_feature = feature_name
                    min_thresh = tresh
                
        print('Min tresh :',  min_thresh,'Min gini :', min_gini,'Min feature :', min_feature)
        return (min_thresh, min_feature, min_gini)   
        
    
    def calculate_gini(self, data_partition_torch, col = '', num_classes = 4):
        """
        Calculates the gini coefficient for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated gini coefficient
        """        
        classQty = data_partition_torch.groupby([data_partition_torch[[col]].columns.item()]).size().array
        totalQty = data_partition_torch.shape[0]
        gini = 1 - sum( (classQty / totalQty) **2)
        
        return gini
   
    
    def evaluate_node(self, input_torch): 
        """
        Evaluates an input observation within the node. 
        If is not a leaf node, send it to the corresponding node
        return predicted label
        """
        feature_val_input = input_torch[self.feature_num]
        if(self.is_leaf()):
            return self.dominant_class
        else:
            if(feature_val_input < self.threshold_value):
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)
        

class CART:
    def __init__(self, dataset_torch, max_CART_depth = 4, min_observations = 2):
        """
        CART has only one root node
        """
        #min observations per node
        self.min_observations = min_observations
        self.root = Node_CART(num_classes = 4, ref_CART = self, current_depth = 0)
        self.max_CART_depth = max_CART_depth
        self.list_selected_features = []
        
    def get_root(self):
        """
        Gets tree root
        """
        return self.root
    
    def get_min_observations(self):
        """
        return min observations per node
        """
        return self.min_observations
    
    def get_max_depth(self):
        """
        Gets the selected max depth of the tree
        """
        return self.max_CART_depth
    
    def build_CART(self, data_torch):
        """
        Build CART from root
        """
        self.list_selected_features = self.root.create_with_children(data_torch, current_depth = 0)
    
    def to_xml(self, xml_file_name):
        """
        write Xml file with tree content
        """
        str_nodes = self.root.to_xml()
        file = open(xml_file_name,"w+") 
        file.write(str_nodes)
        file.close()
        return str_nodes
    
    
    def evaluate_input(self, input_torch):
        """
        Evaluate a specific input in the tree and get the predicted class
        """
        return self.root.evaluate_node(input_torch)
        
    
def train_CART(dataset_torch, name_xml = "", max_CART_depth = 3, min_obs_per_leaf = 2): 
    """
    Train CART model
    """
    tree = CART(dataset_torch = dataset_torch, max_CART_depth = max_CART_depth, min_observations =  min_obs_per_leaf)
    tree.build_CART(dataset_torch)
    if(not name_xml == ""):
        tree.to_xml(name_xml)
    return tree

def test_CART(tree, testset_torch):
    """
    Test a previously built CART
    """
    #TODO, use tree.evaluate_input(current_observation) for this
    return accuracy

        

#tree = train_CART(dataset_torch, name_xml = "CART_example.xml")
#acc = test_CART(tree, dataset_torch)
'''
CART_1 =  CART(dataset_torch)
CART_1.to_xml("arbolito_vacio.xml")
nodo_A = Node_CART(num_classes = 2, current_depth = 1)
CART_1.root.node_left = nodo_A
CART_1.to_xml("arbolito_peque.xml")
'''


dataset_torch = read_dataset()
print(dataset_torch)

CART_DATA = CART(dataset_torch)
CART_DATA.build_CART(dataset_torch)
CART_DATA.to_xml("xmlData.xml")
    
        

      Rooms  Size  Toilets  Parking  Class
0         4   343        7        5      4
1         4   343        6        5      4
2         4   420        6        4      4
3         4   278        7        4      4
4         4   278        5        5      4
...     ...   ...      ...      ...    ...
4889      2    47        1        1      1
4890      2    57        1        1      1
4891      2    56        1        1      1
4892      2    48        1        1      1
4893      2    50        1        1      1

[4894 rows x 5 columns]
Rooms 0 4
Rooms 0 3
Rooms 0 6
Rooms 0 2
Rooms 0 5
Rooms 0 1
Size 1 343
Size 1 420
Size 1 278
Size 1 269
Size 1 279
Size 1 377
Size 1 275
Size 1 415
Size 1 387
Size 1 400
Size 1 302
Size 1 321
Size 1 286
Size 1 319
Size 1 364
Size 1 288
Size 1 540
Size 1 285
Size 1 306
Size 1 316
Size 1 359
Size 1 253
Size 1 307
Size 1 270
Size 1 310
Size 1 292
Size 1 241
Size 1 226
Size 1 201
Size 1 290
Size 1 348
Size 1 232
Size 1 280
Size 1 297
Size 1 300
Size 1 267
Siz

Toilets 2 1
Toilets 2 2
Parking 3 5
Parking 3 4
Parking 3 7
Parking 3 3
Parking 3 6
Min tresh : 4 Min gini : 0.11497726093459086 Min feature : Parking
Parking
Rooms 0 4
Rooms 0 3
Rooms 0 6
Rooms 0 5
Rooms 0 2
Size 1 343
Size 1 420
Size 1 278
Size 1 269
Size 1 279
Size 1 377
Size 1 275
Size 1 415
Size 1 387
Size 1 400
Size 1 302
Size 1 321
Size 1 286
Size 1 319
Size 1 364
Size 1 288
Size 1 540
Size 1 285
Size 1 306
Size 1 316
Size 1 359
Size 1 253
Size 1 307
Size 1 270
Size 1 292
Size 1 226
Size 1 348
Size 1 232
Size 1 267
Size 1 295
Size 1 298
Size 1 265
Size 1 220
Size 1 620
Size 1 250
Size 1 320
Size 1 310
Size 1 230
Size 1 280
Size 1 258
Size 1 257
Size 1 274
Size 1 300
Size 1 206
Size 1 238
Size 1 236
Size 1 294
Size 1 221
Size 1 214
Size 1 237
Size 1 505
Size 1 240
Size 1 255
Size 1 224
Size 1 182
Size 1 233
Size 1 183
Size 1 189
Size 1 218
Size 1 311
Size 1 235
Size 1 225
Size 1 264
Size 1 223
Size 1 191
Size 1 165
Size 1 209
Size 1 227
Size 1 266
Size 1 315
Size 1 161
Size 1 386

TypeError: '(slice(None, None, None), 0)' is an invalid key